In [7]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from PIL import Image
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import random
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from torchvision.transforms import v2



In [18]:
N = 224
N_CLASSES = 2
SMOOTH = 0.1
LOSS = 'CE' # CE or UANLL
SEED = [42, 0, 17, 9, 3]
NUM_TTA_FOR_SEED = 10


In [9]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    pl.seed_everything(seed)
    g = torch.Generator()
    g.manual_seed(seed)
    return g
    
    
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [10]:
class ClassDataset(Dataset):
    def __init__(self, dataframe, preload_transform=None, transform=None):
        self.dataframe = dataframe
        self.preload_transform = preload_transform
        self.transform = transform

        self.images = torch.zeros(len(dataframe), 3, N, N).float()

        for idx in range(len(dataframe)):
            image_path = self.dataframe.index[idx]
            image = Image.open(f'images/{image_path}').convert("RGB")
            if self.preload_transform:
                self.images[idx] = self.preload_transform(image)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if self.transform:
            return self.transform(self.images[idx]), int(self.dataframe.loc[self.dataframe.index[idx], 'CTI_label'])
        else:
            return self.images[idx], int(self.dataframe.loc[self.dataframe.index[idx], 'CTI_label'])


In [11]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc = nn.Sequential(
            nn.Linear(512 * 14 * 14, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 2 if LOSS == 'CE' else 3)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
class UANLLloss(nn.Module):
    def __init__(self, smoothing=SMOOTH, classes=N_CLASSES):
        super(UANLLloss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.clc = classes
        
    def forward(self,x,y):
        logvar  = (x[:,self.clc:]) #** 2 
        prob = x[:,:self.clc]

        with torch.no_grad():
            yoh = torch.zeros_like(prob)
            yoh.fill_(self.smoothing / (self.clc - 1))
            yoh.scatter_(1, y.data.unsqueeze(1), self.confidence)

        loss0 = ((yoh - prob) ** 2).sum(dim=1)
        loss = (torch.exp(-logvar) * loss0 + self.clc * logvar)

        return loss.mean()
    
class ClassModel(pl.LightningModule):
    def __init__(self):
        super(ClassModel, self).__init__()
        self.model = Encoder()
        self.criterion = UANLLloss() if LOSS == 'UANLL' else nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, targets = batch
        outputs = self(images)
        loss = self.criterion(outputs, targets)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        images, targets = batch
        outputs = self(images)
        loss = self.criterion(outputs, targets)
        sm = nn.functional.softmax(outputs, dim=1)
        acc = (sm.argmax(1) == targets).sum() / len(targets)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return {'val_loss': loss, 'val_acc': acc}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3, weight_decay=1e-4)
        # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 16 * 50, gamma=0.5, last_epoch=-1, verbose=0)
        # return [optimizer], [scheduler]
        return optimizer



In [12]:
df = pd.read_csv('total.csv', header=[0, 1], index_col=0)
train_df = df['Kernal'][df['meta']['subset'] == 'train'][['CTI_label', 'LP_label']]
val_df = df['Kernal'][df['meta']['subset'] == 'val'][['CTI_label', 'LP_label']]

transform = transforms.Compose([
    transforms.Resize((N, N)),
    transforms.ToTensor(),
])

train_dataset = ClassDataset(train_df, preload_transform=transform)
val_dataset = ClassDataset(val_df, preload_transform=transform)


In [7]:
checkpoints = []

for seed in SEED:
    g = seed_everything(seed)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, worker_init_fn=seed_worker, generator=g)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        dirpath='checkpoints/encoder',
        filename = f'class_{LOSS}_{seed}_' + '{epoch:02d}-{val_acc:.2f}',
        save_top_k=1,
        mode='min',
    )

    logger = TensorBoardLogger("tb_logs", name=f'class_{LOSS}_{seed}')

    model = ClassModel()

    trainer = pl.Trainer(
        max_epochs=100,
        callbacks=[checkpoint_callback],
        logger=logger,
        deterministic=True
    )

    trainer.fit(model, train_loader, val_loader)
    checkpoints.append(checkpoint_callback.best_model_path)


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
c:\Users\chest\anaconda3\envs\Pytorch\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory E:\Cardiometry with uncertainty\checkpoints\encoder exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params | Mode 
------------------------------------------------
0 | model     | Encoder   | 107 M  | train
1 | criterion | UANLLloss | 0      | train
------------------------------------------------
107 M     Trainable params
0

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\chest\anaconda3\envs\Pytorch\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
c:\Users\chest\anaconda3\envs\Pytorch\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
c:\Users\chest\anaconda3\envs\Pytorch\lib\site-packages\pytorch_lightning\loops\fit_loop.py:298: The number of training batches (16) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [16]:
acc = []
f1 = []
auc = []

test_df = df['Kernal'][df['meta']['subset'] == 'test'][['CTI_label', 'LP_label']]

test_df1 = df['Kernal'][df['meta']['subset'] == 'test'][['CTI_label', 'LP_label']]
test_df2 = df['SuperDoctor'][df['meta']['subset'] == 'test'][['CTI_label', 'LP_label']]
test_df3 = df['Almaz123Almaz123'][df['meta']['subset'] == 'test'][['CTI_label', 'LP_label']]
test_df4 = df['Albert123Albert123'][df['meta']['subset'] == 'test'][['CTI_label', 'LP_label']]

mean_labels = test_df1['CTI_label'] + test_df2['CTI_label'] + test_df3['CTI_label'] + test_df4['CTI_label']

test_df['CTI_label'][mean_labels >= 2] = 1
test_df['CTI_label'][mean_labels < 2] = 0

transform_tta = v2.Compose([
    v2.RandomResizedCrop(N, scale=(0.9, 1), antialias=True)
    ])

In [22]:
print(train_df['CTI_label'].value_counts(normalize=True))
print(val_df['CTI_label'].value_counts(normalize=True))
print(test_df['CTI_label'].value_counts(normalize=True))

0.0    0.560976
1.0    0.439024
Name: CTI_label, dtype: float64
0.0    0.545455
1.0    0.454545
Name: CTI_label, dtype: float64
0.0    0.581818
1.0    0.418182
Name: CTI_label, dtype: float64


# No TTA

In [29]:
test_dataset = ClassDataset(test_df, preload_transform=transform)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for checkpoint in checkpoints:
    model = ClassModel()
    model.load_state_dict(torch.load(checkpoint)['state_dict'])
    model.to('cuda')

    labels_all = []
    outputs_sm_all = []
    predictions_all = []

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_loader):
            inputs = inputs.to('cuda')
            labels = labels.to('cuda')

            outputs = model(inputs)
            outputs_sm = nn.functional.softmax(outputs, dim=1)

            labels_all.append(labels.cpu())
            outputs_sm_all.append(outputs_sm.cpu())
            predictions_all.append(outputs_sm.argmax(1).cpu())

    labels_all = torch.cat(labels_all).numpy()
    outputs_sm_all = torch.cat(outputs_sm_all).numpy()
    predictions_all = torch.cat(predictions_all).numpy()

    accuracy_score_test = accuracy_score(labels_all, predictions_all)
    f1_score_test = f1_score(labels_all, predictions_all)
    roc_auc_score_test = roc_auc_score(labels_all, outputs_sm_all[:, 1])

    acc.append(accuracy_score_test)
    f1.append(f1_score_test)
    auc.append(roc_auc_score_test)

    print(f'acc {accuracy_score_test}, f1 {f1_score_test}, aucroc {roc_auc_score_test}')

print('acc')
print(f'mean {np.mean(acc)}, std {np.std(acc)}')
print('f1')
print(f'mean {np.mean(f1)}, std {np.std(f1)}')
print('aucroc')
print(f'mean {np.mean(auc)}, std {np.std(auc)}')

acc 0.7757575757575758, f1 0.7218045112781954, aucroc 0.836503623188406
acc 0.7696969696969697, f1 0.7205882352941176, aucroc 0.8271437198067633
acc 0.8121212121212121, f1 0.7633587786259542, aucroc 0.8714522946859903
acc 0.7636363636363637, f1 0.7194244604316546, aucroc 0.8248792270531401
acc 0.7818181818181819, f1 0.7391304347826086, aucroc 0.7876660628019324
acc
mean 0.7767272727272727, std 0.01646701352508286
f1
mean 0.7294149435617271, std 0.015959463647868242
aucroc
mean 0.804773902979066, std 0.04176270464368496


# TTA

In [30]:
test_dataset = ClassDataset(test_df, preload_transform=transform, transform=transform_tta)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for checkpoint in checkpoints:
    model = ClassModel()
    model.load_state_dict(torch.load(checkpoint)['state_dict'])
    model.to('cuda')

    labels_all = []
    outputs_sm_all = []

    first = True

    for seed in SEED:
        seed_everything(seed)

        for j in range(NUM_TTA_FOR_SEED):

            with torch.no_grad():
                for i, (inputs, labels) in enumerate(test_loader):
                    inputs = inputs.to('cuda')
                    labels = labels.to('cuda')

                    outputs = model(inputs)
                    outputs_sm = nn.functional.softmax(outputs, dim=1)

                    if first:
                        labels_all.append(labels.cpu())
                        outputs_sm_all.append(outputs_sm.cpu())
                    else:
                        outputs_sm_all[i] += outputs_sm.cpu()
        first = False

    labels_all = torch.cat(labels_all).numpy()
    outputs_sm_all = torch.cat(outputs_sm_all).numpy() / (len(SEED) * NUM_TTA_FOR_SEED)
    predictions_all = outputs_sm_all.argmax(1)

    accuracy_score_test = accuracy_score(labels_all, predictions_all)
    f1_score_test = f1_score(labels_all, predictions_all)
    roc_auc_score_test = roc_auc_score(labels_all, outputs_sm_all[:, 1])

    acc.append(accuracy_score_test)
    f1.append(f1_score_test)
    auc.append(roc_auc_score_test)

    print(f'acc {accuracy_score_test}, f1 {f1_score_test}, aucroc {roc_auc_score_test}')

print('acc')
print(f'mean {np.mean(acc)}, std {np.std(acc)}')
print('f1')
print(f'mean {np.mean(f1)}, std {np.std(f1)}')
print('aucroc')
print(f'mean {np.mean(auc)}, std {np.std(auc)}')


Seed set to 42
Seed set to 0
Seed set to 17
Seed set to 9
Seed set to 3


acc 0.7739393939393939, f1 0.7210172026925954, aucroc 0.756436443236715


Seed set to 42
Seed set to 0
Seed set to 17
Seed set to 9
Seed set to 3


acc 0.7509090909090909, f1 0.705798138869005, aucroc 0.7481544384057972


Seed set to 42
Seed set to 0
Seed set to 17
Seed set to 9
Seed set to 3


acc 0.786060606060606, f1 0.7379361544172235, aucroc 0.7719633152173913


Seed set to 42
Seed set to 0
Seed set to 17
Seed set to 9
Seed set to 3


acc 0.7593939393939394, f1 0.7170349251603706, aucroc 0.7394738828502416


Seed set to 42
Seed set to 0
Seed set to 17
Seed set to 9
Seed set to 3


acc 0.7745454545454545, f1 0.7308248914616498, aucroc 0.7602906099033817
acc
mean 0.7747878787878788, std 0.015904212816345922
f1
mean 0.7276917733013375, std 0.015194275637111024
aucroc
mean 0.7923963617149758, std 0.04240250368569385
